In [1]:
%load_ext autoreload
%autoreload 2

from logging import log
import os
import ray
import time
import warnings
from algorithms.utils import Config, LogClient, LogServer
from algorithms.envs.Flow import makeFlowGrid, makeFlowGridTest
from algorithms.config.FLOW_PPO import getArgs
from algorithms.mbdppo.MB_DPPO import OnPolicyRunner, DPPOAgent
import torch




In [1]:
import torch
t = torch.load('checkpoints/standard_makeFigureEight2_MB_DPPOAgent_17361/81501_5222.7847817614875.pt')

In [3]:
print(t[''].keys())

dict_keys(['actors.0.log_std', 'actors.0.network.0.weight', 'actors.0.network.0.bias', 'actors.0.network.2.weight', 'actors.0.network.2.bias', 'actors.0.network.4.weight', 'actors.0.network.4.bias', 'actors.0.action_head.weight', 'actors.0.action_head.bias', 'actors.1.log_std', 'actors.1.network.0.weight', 'actors.1.network.0.bias', 'actors.1.network.2.weight', 'actors.1.network.2.bias', 'actors.1.network.4.weight', 'actors.1.network.4.bias', 'actors.1.action_head.weight', 'actors.1.action_head.bias', 'actors.2.log_std', 'actors.2.network.0.weight', 'actors.2.network.0.bias', 'actors.2.network.2.weight', 'actors.2.network.2.bias', 'actors.2.network.4.weight', 'actors.2.network.4.bias', 'actors.2.action_head.weight', 'actors.2.action_head.bias', 'actors.3.log_std', 'actors.3.network.0.weight', 'actors.3.network.0.bias', 'actors.3.network.2.weight', 'actors.3.network.2.bias', 'actors.3.network.4.weight', 'actors.3.network.4.bias', 'actors.3.action_head.weight', 'actors.3.action_head.bias

In [2]:

warnings.filterwarnings('ignore')

def getRunArgs():
    run_args = Config()
    run_args.n_thread = 1
    run_args.parallel = False
    run_args.device = 'cpu'
    run_args.n_cpu = 1/4
    run_args.n_gpu = 0
    run_args.debug = True
    run_args.test = False
    run_args.profiling = False
    run_args.name = 'standard'
    run_args.radius_v = 3
    run_args.radius_pi = 1
    run_args.radius_p = 1
    run_args.init_checkpoint = None
    run_args.start_step = 0
    run_args.save_period = 1800 # in seconds
    run_args.log_period = int(20)
    run_args.seed = None
    return run_args

def initArgs(run_args, env_train, env_test):
    ref_env = env_train
    alg_args = getArgs(run_args.radius_p, run_args.radius_v, run_args.radius_pi, ref_env)
    return alg_args

def initAgent(logger, device, agent_args):
    return DPPOAgent(logger, device, agent_args)

def override(alg_args, run_args, env_fn_train):
    alg_args.env_fn = env_fn_train
    agent_args = alg_args.agent_args
    p_args, v_args, pi_args = agent_args.p_args, agent_args.v_args, agent_args.pi_args
    if run_args.debug:
        alg_args.model_batch_size = 4
        alg_args.max_ep_len=20
        alg_args.rollout_length = 20
        alg_args.test_length = 1
        alg_args.model_buffer_size = 4
        alg_args.n_warmup=1
        alg_args.n_test=1
    if run_args.test:
        alg_args.n_warmup = 0
        alg_args.n_test = 10
    if run_args.profiling:
        alg_args.model_batch_size = 128
        alg_args.n_warmup = 0
        if alg_args.agent_args.p_args is None:
            alg_args.n_iter = 2
        else:
            alg_args.n_iter = 2
            alg_args.model_buffer_size = 1000
            alg_args.n_warmup = 1
        alg_args.n_test = 1
        alg_args.max_ep_len = 20
        alg_args.rollout_length = 20
        alg_args.test_length = 20
    if run_args.seed is None:
        run_args.seed = int(time.time()*1000)%65536
    agent_args.parallel = run_args.parallel
    return alg_args, run_args

env_fn_train, _ = makeFlowGrid()
env_fn_test, _ = makeFlowGridTest()
env_train = env_fn_train()
env_test = env_fn_test()
run_args = getRunArgs()
alg_args = initArgs(run_args, env_train, env_test)
alg_args, run_args = override(alg_args, run_args, env_fn_train)

os.environ['CUDA_VISIBLE_DEVICES']='0'
ray.init(ignore_reinit_error = True, num_gpus=len(os.environ['CUDA_VISIBLE_DEVICES'].split(',')))
logger = LogServer.remote({'run_args':run_args, 'algo_args':alg_args}, mute=run_args.debug or run_args.test or run_args.profiling)
logger = LogClient(logger)
agent = initAgent(logger, run_args.device, alg_args.agent_args)

torch.set_num_threads(run_args.n_thread)
print(f"n_threads {torch.get_num_threads()}")
print(f"n_gpus {torch.cuda.device_count()}")

2021-08-30 10:10:19,720	WARNING utils.py:518 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.


n_threads 1
n_gpus 0


In [3]:

if run_args.profiling:
    import cProfile
    cProfile.run("OnPolicyRunner(logger = logger, run_args=run_args, alg_args=alg_args, agent=agent, env_learn=env_train, env_test = env_test).run()",
                 filename=f'device{run_args.device}_parallel{run_args.parallel}.profile')
else:
    OnPolicyRunner(logger = logger, run_args=run_args, alg_args=alg_args, agent=agent, env_learn=env_train, env_test = env_test).run()


  0%|          | 0/1 [00:01<?, ?it/s]


RuntimeError: size mismatch, m1: [1 x 378], m2: [42 x 64] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:136